In [9]:
import random
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()



# Define the list of picture coordinates


# Randomly select initial coordinates for each picture

# take the window size
window_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))


text = 'Rectangle Game'
text_size= cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

radius = 4
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]


touching = False
index_finger_x_coordinates = []
index_finger_y_coordinates = []



while True:
    success, image = cap.read()

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # Here the image arrray is read only mode
    image.flags.writeable = False
    results = hands.process(image)
    # print(results)
    # Here the image arrray is writeable mode
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            

            # Check if index finger is touching any picture
            index_finger_coordinates = (
                hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image.shape[1],
                hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image.shape[0]
            )
            thumb_coordinates = (
                hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image.shape[1],
                hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image.shape[0]
            )
            middle_finger_coordinates = (
                hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image.shape[1],
                hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image.shape[0]
            )
            if abs(index_finger_coordinates[0] - thumb_coordinates[0]) < 15 and abs(index_finger_coordinates[1] - thumb_coordinates[1]) < 15:
                if abs(index_finger_coordinates[0] - middle_finger_coordinates[0]) < 15 and abs(index_finger_coordinates[1] - middle_finger_coordinates[1]) < 15:
                    if index_finger_coordinates < (50,  50):
                        index_finger_x_coordinates = []
                        index_finger_y_coordinates = []

                touching = True
                index_finger_x_coordinates.append(index_finger_coordinates[0])
                index_finger_y_coordinates.append(index_finger_coordinates[1])

                
            
            else:
                touching = False
    
    for i,j in zip(index_finger_x_coordinates, index_finger_y_coordinates):
        cv2.circle(image, (int(i), int(j)), radius, random.choice(colors), -1)
            
        # Display the points counter
    cv2.imshow("Games", image)
    if cv2.waitKey(5) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()
